## About

For this notebook, we have two goals.

1) get familiar with invoking different LLMs using Langchain.
2) Explore the response of LLM's for the query rewriting use case


## Import

In [141]:
from dotenv import load_dotenv
import os

import os
from langchain.chat_models import ChatOpenAI, ChatGooglePalm
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.llms import Replicate

from langchain.schema import StrOutputParser
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache
from langchain.callbacks import get_openai_callback

from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain.output_parsers import CommaSeparatedListOutputParser
import tiktoken
from langchain.llms import CTransformers
from langchain.llms.huggingface_pipeline import HuggingFacePipeline 
from langchain.llms import HuggingFaceHub
from pydantic import BaseModel, Field

import rich

In [2]:
from langchain.globals import set_debug


In [3]:
set_debug(True)


In [4]:
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION']="python"


In [5]:
load_dotenv("../.env") 

True

to save time and cost, we make sure to save the LLM output

In [25]:
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


## Use Case: Reasoning

### LLM: OpenAI

All the availalbe OpenAI models can be found [here](https://platform.openai.com/docs/models/continuous-model-upgrades) 

Here we are using Gpt4, that supports 8k tokens

In [7]:
llm_openai= ChatOpenAI(model="gpt-4")
#llm_openai = ChatOpenAI(model="gpt-3.5-turbo")


In [8]:
template = """Question: {question}


Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

In [9]:
question = "Can Barack Obama have a conversation with George Washington?"

In [10]:
prompt.format(question=question)

'Question: Can Barack Obama have a conversation with George Washington?\n\n\nAnswer: '

In [11]:
chain_gpt4 = LLMChain(llm=llm_openai, prompt=prompt)


In [12]:
with get_openai_callback() as cb:
    result = chain_gpt4.run(question="Can Barack Obama have a conversation with George Washington?")

    print(cb)
    print(result)

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "question": "Can Barack Obama have a conversation with George Washington?"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Question: Can Barack Obama have a conversation with George Washington?\n\n\nAnswer:"
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:ChatOpenAI] [2.12s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "No, Barack Obama cannot have a conversation with George Washington because George Washington died in 1799, long before Obama was born.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "No, Barack Obama cannot have 

understanding openai tokenizer

In [13]:
encoding = tiktoken.encoding_for_model('gpt-4')

In [14]:
message = prompt.format(question=question)
token_integers = encoding.encode(message )

num_tokens = len(token_integers)
token_bytes = [encoding.decode_single_token_bytes(token) for token in token_integers]

print (token_integers)

print (f"num tokens: {num_tokens}")

print (token_bytes)


[14924, 25, 3053, 24448, 7250, 617, 264, 10652, 449, 10058, 6652, 31931, 16533, 25, 220]
num tokens: 15
[b'Question', b':', b' Can', b' Barack', b' Obama', b' have', b' a', b' conversation', b' with', b' George', b' Washington', b'?\n\n\n', b'Answer', b':', b' ']


The discrepancy in tokens, is because there is a `role` added to the message 

### LLM: Google Palm

In [15]:
llm_palm = ChatGooglePalm()

In [16]:
chain_palm = LLMChain(llm=llm_palm, prompt=prompt, output_parser=StrOutputParser())


In [17]:
response = chain_palm.run(question=question)


[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "question": "Can Barack Obama have a conversation with George Washington?"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatGooglePalm] Entering LLM run with input:
{
  "prompts": [
    "Human: Question: Can Barack Obama have a conversation with George Washington?\n\n\nAnswer:"
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:ChatGooglePalm] [10.57s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Barack Obama cannot have a conversation with George Washington. George Washington died in 1799, while Barack Obama was born in 1961. Therefore, they were never alive at the same time.\r\n\r\nHowever, it is possible for Barack Obama to have a conversation with a digital representation of George Washington. This could be done through a computer program that uses artificial intelligence to generate text and speech that is indistinguishable from a human. Such a program could be trained on a large corpus of te

In [18]:
print (response)

Barack Obama cannot have a conversation with George Washington. George Washington died in 1799, while Barack Obama was born in 1961. Therefore, they were never alive at the same time.

However, it is possible for Barack Obama to have a conversation with a digital representation of George Washington. This could be done through a computer program that uses artificial intelligence to generate text and speech that is indistinguishable from a human. Such a program could be trained on a large corpus of text and speech from George Washington, and it could be used to create a realistic simulation of his personality and mannerisms.

It is also possible for Barack Obama to have a conversation with a historical figure through a medium such as a Ouija board or a seance. However, there is no scientific evidence to support the existence of such phenomena, and it is therefore more likely that any such conversation would be a product of the imagination of the participants.


### LLM Provider: Huggingface

you can use models from hugginface too.   



In [19]:


llm_zephyr = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-alpha", model_kwargs={"temperature": 0.1}
)

/usr/local/lib/python3.9/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [20]:
chain_zephyr = LLMChain(prompt=prompt, llm=llm_zephyr)



In [21]:
response = chain_zephyr.run(question="Can Barack Obama have a conversation with George Washington?")


[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "question": "Can Barack Obama have a conversation with George Washington?"
}
[llm/start] [1:chain:LLMChain > 2:llm:HuggingFaceHub] Entering LLM run with input:
{
  "prompts": [
    "Question: Can Barack Obama have a conversation with George Washington?\n\n\nAnswer:"
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:HuggingFaceHub] [1.50s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n\nNo, Barack Obama cannot have a conversation with George Washington because George Washington is deceased and has been for over 200 years. Conversations can only take place between living beings.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:LLMChain] [1.50s] Exiting Chain run with output:
{
  "text": "\n\nNo, Barack Obama cannot have a conversation with George Washington because George Washington is deceased and ha

if, you want to do only local inference

In [22]:
# llm_zephyr = HuggingFacePipeline.from_model_id(
#     model_id="HuggingFaceH4/zephyr-7b-alpha",
#     task="text-generation",
#     pipeline_kwargs={},
# )

### LLM Provider: Replicate

https://python.langchain.com/docs/integrations/llms/replicate

exploring different Llama models from facebook

In [23]:

llm_llama7b = Replicate(
    model="meta/llama-2-7b:77dde5d6c56598691b9008f7d123a18d98f40e4b4978f8a72215ebfc2553ddd8",
    verbose=True
)


llm_llama7b_chat = Replicate(
    model="meta/llama-2-7b-chat:13c3cdee13ee059ab779f0291d29054dab00a47dad8261375654de5540165fb0",
    verbose=True
)


llm_llama13b = Replicate(
    model="meta/llama-2-13b:078d7a002387bd96d93b0302a4c03b3f15824b63104034bfa943c63a8f208c38",
    verbose=True
)



llm_llama13b_chat = Replicate(
    model="meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d",
    verbose=True
)


llm_llama70b = Replicate(
    model="meta/llama-2-70b:a52e56fee2269a78c9279800ec88898cecb6c8f1df22a6483132bea266648f00",
    verbose=True
)


llm_llama70b_chat = Replicate(
    model="meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    verbose=True
)




llama7b base model

In [29]:
print ( llm_llama7b(prompt.format(question=question) ))

[llm/start] [1:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Question: Can Barack Obama have a conversation with George Washington?\n\n\nAnswer:"
  ]
}
[llm/end] [1:llm:Replicate] [2.54s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1) \\strong{No.} - If we are talking about the actual person who founded America and served as its first president, then no. He died in 1799; Obama was born in 1964.\n\n2) \\strong{Yes.} - The term \"Founding Father\" is applied to people like Adams or Jefferson (born ca. 1735-1740), so they could conceivably meet. However, I would think that, if there were any conversations between these two men, it's unlikely either of them mentioned their roles as Founders",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
1) \strong{No.} - If we are talking about the actual person who founded America and served as its first president, then no. He

llama7b chat model

In [30]:
print ( llm_llama7b_chat(prompt.format(question=question) ))

[llm/start] [1:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Question: Can Barack Obama have a conversation with George Washington?\n\n\nAnswer:"
  ]
}
[llm/end] [1:llm:Replicate] [1.60s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Oh, what an interesting question! *adjusts glasses* I must respectfully point out that Barack Obama and George Washington lived in different time periods, making it impossible for them to have a conversation with each other.\n\nGeorge Washington was born in 1732 and passed away in 1799, while Barack Obama was born in 1961 and served as the 44th President of the United States from 2009 to 2017. This means that they lived over 200 years apart, and there is no recorded",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
Oh, what an interesting question! *adjusts glasses* I must respectfully point out that Barack Obama and George Washingt

llama13b base model

In [31]:
print ( llm_llama13b(prompt.format(question=question) ))

[llm/start] [1:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Question: Can Barack Obama have a conversation with George Washington?\n\n\nAnswer:"
  ]
}
[llm/end] [1:llm:Replicate] [34.78s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n\n\\begin{blockquote}\n\nNo. They both lived at different times and died in different places (Washington died in Virginia, while Obama is still alive). So they can't be having any conversations together!\n\\end{blockquote}",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}


\begin{blockquote}

No. They both lived at different times and died in different places (Washington died in Virginia, while Obama is still alive). So they can't be having any conversations together!
\end{blockquote}


llama13b chat model

In [32]:
print ( llm_llama13b_chat(prompt.format(question=question) ))

[llm/start] [1:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Question: Can Barack Obama have a conversation with George Washington?\n\n\nAnswer:"
  ]
}
[llm/end] [1:llm:Replicate] [6.84s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Ah, an intriguing question indeed! While it may not be possible for Barack Obama to have a direct conversation with George Washington, as they lived in different time periods and were separated by centuries, I can certainly imagine a thought-provoking dialogue between these two remarkable individuals.\n\nIn this hypothetical conversation, Barack Obama might ask George Washington about his experiences as the first president of the United States, including his views on leadership, governance, and the challenges he faced during his time in office. George Washington, with his wisdom and insight gained from leading the Continental Army and shaping",
        "generation_info": null,
        "type": "Generat

llama70b base model

In [34]:
print ( llm_llama70b(prompt.format(question=question) ))

[llm/start] [1:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Question: Can Barack Obama have a conversation with George Washington?\n\n\nAnswer:"
  ]
}
[llm/end] [1:llm:Replicate] [11.59s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1.Barack can speak to his father, and Barack's father can speak to his father...\n2. ...and so on until you get back to the first person who could speak to Washington. (I'm not sure how far back that would be.) Then it is just a matter of going up through the line from there, like in #1.\n\nComment: Why do we need to go back all the way to the first ancestor who could talk to Washington? That seems to make this problem equivalent to the \"ancestors\" problem, which has a known solution - I think. If my",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
1.Barack can speak to his father, and Barack's father can speak to his father...


llama70b chat model

In [33]:
print ( llm_llama70b_chat(prompt.format(question=question) ))

[llm/start] [1:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Question: Can Barack Obama have a conversation with George Washington?\n\n\nAnswer:"
  ]
}
[llm/end] [1:llm:Replicate] [21.01s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " I'm happy to help! While it's not possible for Barack Obama to have a direct conversation with George Washington, as they lived in different time periods, it's interesting to consider what their conversation might look like if they were able to communicate across time.\n\nBarack Obama was the first African American President of the United States, serving two terms from 2009 to 2017. He is known for his efforts to reform healthcare, address climate change, and promote civil rights.\n\nGeorge Washington, on the other hand, was the first President of the United",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
 I'm happy to help! Whi

## Use Case: Ecommerce Doc2Query

How do we deal with the the vocabulary mismatch ; where the tokens used in the query and Item are different ?

How do we avoid stuffing inside the item title ?




One technique is Doc2Query

![stuff](https://i0.wp.com/sease.io/wp-content/uploads/2021/12/dco2query.png?resize=2048%2C1351&ssl=1)


Image Ref: [Sease](https://sease.io/2022/01/tackling-vocabulary-mismatch-with-document-expansion.html)


In [90]:
set_debug(False)

In [91]:
sample_product_title = "Biodegradable Bamboo Toothbrush, Natural Charcoal toothbrushes Soft Bristle Toothbrush Eco-Friendly Sustainable Toothbrush BPA Free Organic Compostable Travel Toothbrushes Wooden toothbrushes, 6 Pack"


### Zero Shot prompting

In [92]:
generic_zero_shot_template = """
Given the title of a product, generate two possible user search queries. 

Ideally prioritize queries with tokens not in the product title 


Input:   
Title: {product_title}
Output: 


"""

In [93]:
generic_zero_shot_prompt = PromptTemplate(template=generic_zero_shot_template, input_variables=["product_title"]
                                          )
                                 

In [94]:
print ( generic_zero_shot_prompt.format(product_title = sample_product_title ) )



Given the title of a product, generate two possible user search queries. 

Ideally prioritize queries with tokens not in the product title 


Input:   
Title: Biodegradable Bamboo Toothbrush, Natural Charcoal toothbrushes Soft Bristle Toothbrush Eco-Friendly Sustainable Toothbrush BPA Free Organic Compostable Travel Toothbrushes Wooden toothbrushes, 6 Pack
Output: 





In [95]:
chain = generic_zero_shot_prompt | llm_openai 
chain.invoke({"product_title": sample_product_title})

AIMessage(content='1. "Eco friendly toothbrushes pack of 6"\n2. "Compostable travel toothbrushes with soft bristles"')

it can be hard to parse this response, lets use langchain format parser, to help format the response

In [96]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

Here is the prompt with auto generated format instructions

In [152]:
generic_zero_shot_template = """
Given the title of a product, generate two possible user search queries. 

Ideally prioritize queries with tokens not in the product title 


Input:   
Title: {product_title}
Output: 

Format Instructions:
{format_instructions}

"""

In [153]:
generic_zero_shot_prompt = PromptTemplate(template=generic_zero_shot_template, input_variables=["product_title"] 
                                         , partial_variables={"format_instructions": format_instructions} 
                                         )


In [154]:
print ( generic_zero_shot_prompt.format(product_title = sample_product_title ) )



Given the title of a product, generate two possible user search queries. 

Ideally prioritize queries with tokens not in the product title 


Input:   
Title: Biodegradable Bamboo Toothbrush, Natural Charcoal toothbrushes Soft Bristle Toothbrush Eco-Friendly Sustainable Toothbrush BPA Free Organic Compostable Travel Toothbrushes Wooden toothbrushes, 6 Pack
Output: 

Format Instructions:
Your response should be a list of comma separated values, eg: `foo, bar, baz`




In [100]:
chain = generic_zero_shot_prompt | llm_openai | output_parser
chain.invoke({"product_title": sample_product_title})

['"Eco-friendly toothbrush pack', 'Natural wooden toothbrushes for travel"']

output looks almost right , but there is some extra quote 😔

### Role based prompting

In [156]:
ecommerce_zero_shot_template = """
You are an e-commerce shopping assistant. 
Given the title of a product, generate two possible user search queries. 
Ideally prioritize queries with tokens not in the product title 

{format_instructions}

Input:   
Title: {product_title}
Output: 


"""
ecommerce_zero_shot_prompt = PromptTemplate(template=ecommerce_zero_shot_template, input_variables=["product_title"],
                                 partial_variables={"format_instructions": format_instructions}
                                 )





In [157]:
chain = ecommerce_zero_shot_prompt | llm_openai | output_parser
chain.invoke({"product_title": sample_product_title})

['eco friendly dental care', 'compostable toothbrushes pack']

it seems, by priming the agent to be an e-commerce assistant, it outputed the response in a friendly manner

lets be explicit about the format instructions

In [158]:

ecommerce_template = """
You are an e-commerce shopping assistant. 
Given the title of a product, generate two possible user search queries. 
Ideally prioritize queries with tokens not in the product title 

{format_instructions}

Examples:   
Input:   
Title: Let's Find Pokémon! Special Complete Edition (2nd edition)   
Output:    
pokemon book pokedex   
pokemon illustration book   
Input:   
Title: Keto Comfort Foods: Family Favorite Recipes Made Low-Carb and Healthy   
Output:   
soul food cookbook for beginners   
chicken soup for nutrient 

Input:   
Title: {product_title}
Output: 


"""
ecommerce_prompt = PromptTemplate(template=ecommerce_template, input_variables=["product_title"],
                                 partial_variables={"format_instructions": format_instructions}
                                 )





In [159]:
chain = ecommerce_prompt | llm_openai | output_parser

chain.invoke({"product_title": sample_product_title})

['eco friendly oral care products', 'sustainable dental hygiene items']

parsed it correctly 🎉

### Llama Prompt format

Llama chat models, were trained with the below prompt format

```
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>

{{ user_message }} [/INST]
```


In [106]:
ecommerce_sys_template = """

<s>[INST] <<SYS>> You are an e-commerce shopping assistant. <</SYS>>

Given the title of a product, generate two possible user search queries. 
Ideally prioritize queries with tokens not in the product title. 
{format_instructions}

Few shot Examples:   
Input:   
Title: Let's Find Pokémon! Special Complete Edition (2nd edition)   
Output:    
pokemon book pokedex   
pokemon illustration book   
Input:   
Title: Keto Comfort Foods: Family Favorite Recipes Made Low-Carb and Healthy   
Output:   
soul food cookbook for beginners   
chicken soup for nutrient 

Input:   
Title: {product_title} 
Output: 
[/INST]


"""



ecommerce_sys_prompt = PromptTemplate(template=ecommerce_sys_template, input_variables=["product_title"],
                                 partial_variables={"format_instructions": format_instructions}
                                 )

In [107]:
print ( ecommerce_sys_prompt.format(product_title = sample_product_title ) )



<s>[INST] <<SYS>> You are an e-commerce shopping assistant. <</SYS>>

Given the title of a product, generate two possible user search queries. 
Ideally prioritize queries with tokens not in the product title. 
Your response should be a list of comma separated values, eg: `foo, bar, baz`

Few shot Examples:   
Input:   
Title: Let's Find Pokémon! Special Complete Edition (2nd edition)   
Output:    
pokemon book pokedex   
pokemon illustration book   
Input:   
Title: Keto Comfort Foods: Family Favorite Recipes Made Low-Carb and Healthy   
Output:   
soul food cookbook for beginners   
chicken soup for nutrient 

Input:   
Title: Biodegradable Bamboo Toothbrush, Natural Charcoal toothbrushes Soft Bristle Toothbrush Eco-Friendly Sustainable Toothbrush BPA Free Organic Compostable Travel Toothbrushes Wooden toothbrushes, 6 Pack 
Output: 
[/INST]





In [160]:
chain = ecommerce_prompt | llm_openai | output_parser

print ( chain.invoke({"product_title": sample_product_title}) )

['eco friendly oral care products', 'sustainable dental hygiene items']


google palm model

In [109]:
chain = ecommerce_prompt | llm_palm | output_parser

chain.invoke({"product_title": sample_product_title})

['Here are two possible user search queries for the product title "Biodegradable Bamboo Toothbrush',
 'Natural Charcoal toothbrushes Soft Bristle Toothbrush Eco-Friendly Sustainable Toothbrush BPA Free Organic Compostable Travel Toothbrushes Wooden toothbrushes',
 '6 Pack":\n\n* Bamboo toothbrush\n* Eco-friendly toothbrush\n\nI prioritized queries with tokens not in the product title',
 'such as "bamboo" and "eco-friendly." I also considered the product\'s features',
 'such as its biodegradable and sustainable nature.']



it doesn't return the format in the correct output.   
It also provided explanations

### evaluate different llama models

llm_llama7b

In [161]:
chain = ecommerce_prompt | llm_llama7b.bind( )

print ( chain.invoke({"product_title": sample_product_title}) )

The following is your test data:  

```sh
$ cat test_data/inputs
title1
title2
title3
title4
title5
title6
title7
title8
title9
title10
title11
title12
title13
title14
title15
title16
title17
title18
title19
title20
title21
title22
title23
title24
title25
title26
title27
title28
title29


it seems to treat the prompt as code completion

llm_llama7b_chat

In [129]:
chain = ecommerce_prompt | llm_llama7b_chat.bind( ) 

print ( chain.invoke({"product_title": sample_product_title}) )

Understood! Here are two possible user search queries for each product:

For "Let's Find Pokémon! Special Complete Edition (2nd edition)":

* pokemon books for kids
* complete pokemon encyclopedia

For "Keto Comfort Foods: Family Favorite Recipes Made Low-Carb and Healthy":

* soul food cookbook vegan
* low-carb comfort food recipes
* healthy family dinner ideas

For "Biodegradable Bamboo Toothbrush, Natural Charcoal to


It generates queries for examples already provided.  
It generates three queries for the second one.  

It doesn't provide examples for the actual one we are interested in


what if we used the prompt format, that it was trained on ? 

In [139]:
chain = ecommerce_sys_prompt | llm_llama7b_chat.bind( ) 

print ( chain.invoke({"product_title": sample_product_title}) )

Understood! Here are two possible user search queries for the product title "Let's Find Pokémon! Special Complete Edition (2nd edition)":

1. `pokemon books for kids` - This query prioritizes tokens not in the product title, such as "kids".
2. `gift ideas for pokémon fans` - This query is more focused on a specific use case or occasion, which can help users find related products more easily.


llm_llama13b

In [130]:
chain = ecommerce_prompt | llm_llama13b.bind( ) 

print ( chain.invoke({"product_title": sample_product_title}) )

### Solution 1 - Stacking Tokens
![](https://github.com/JayWang94/CodingInterviewChallenges/blob/master/Solutions/LeetCode%20Solutions/08_Stack/StackQuery.png?raw=true)
```java
class Solution {
   public List<String> findSearchTerms(String title) {
       if (title == null || title.length() < 3) return new ArrayList<>();
       
       int start = 0;
       while ((start + 1) <= title


again the base model, seems to want to generate code

In [132]:
chain = ecommerce_prompt | llm_llama13b_chat.bind( ) 

print ( chain.invoke({"product_title": sample_product_title}) )

 Sure, I'd be happy to help! Here are two possible user search queries based on the product title you provided:

* "eco-friendly toothbrushes" (this query includes the token "eco-friendly" which is not present in the product title)
* "travel toothbrushes" (this query includes the token "travel" which is not present in the product title)

So the output would be:

eco-friendly toothbrushes, travel toothbrushes


In [138]:
chain = ecommerce_sys_prompt | llm_llama13b_chat.bind( ) 

print ( chain.invoke({"product_title": sample_product_title}) )

 Sure, I'd be happy to help! Here are two possible user search queries based on the given product title:

* "biodegradable toothbrushes" (token not in the title: "biodegradable")
* "natural charcoal toothbrushes" (tokens not in the title: "natural", "charcoal")


The 13b model generated the output and explanation, regardless of the prompt

llama 70 chat

In [163]:
chain = ecommerce_prompt | llm_llama70b.bind() 

print ( chain.invoke({"product_title": sample_product_title}) )

### Expected Response
The expected response is a list of strings, where each string represents one query suggestion, as specified by the problem statement above.  

Example responses:   
```json
[ "search term", "search term" ]
```



In [164]:
chain = ecommerce_zero_shot_prompt | llm_llama70b_chat.bind() 

print ( chain.invoke({"product_title": sample_product_title}) )

 Sure! Here are two possible user search queries for the given product title:

1. "Eco-friendly toothbrushes" - This query prioritizes the token "eco-friendly" which is not present in the product title, but is highly relevant to the product's biodegradable and sustainable features.
2. "Natural charcoal toothbrush" - This query prioritizes the token "natural charcoal" which is also not present in the product title, but is a key feature of the product's bristles.



zero shoting the 70b chat model produces good results, but not in the format

In [165]:
chain = ecommerce_prompt | llm_llama70b_chat.bind() 

print ( chain.invoke({"product_title": sample_product_title}) )

 Sure, here are two possible user search queries for each product:

1. Title: Let's Find Pokémon! Special Complete Edition (2nd edition)
	* Search query 1: "pokemon guide"
	* Search query 2: "illustrated pokemon encyclopedia"
2. Title: Keto Comfort Foods: Family Favorite Recipes Made Low-Carb and Healthy
	* Search query 1: "keto diet recipe book"
	* Search query 2: "low carb comfort food recipes


not sure why the 70b model is misbehaving this time 

In [166]:
chain = ecommerce_sys_prompt | llm_llama70b_chat.bind( ) 

print ( chain.invoke({"product_title": sample_product_title}) )

 Sure, here are two possible user search queries for the given product title:

1. "eco-friendly toothbrush"
2. "compostable toothbrush"

Explanation:

* The first query, "eco-friendly toothbrush," is a more general search term that prioritizes the environmental benefits of the product. This query could be used by users who are looking for sustainable alternatives to traditional toothbrushes.
* The second query, "compostable toothbrush," is more specific and highlights


the results are similar to the zero shot 70b chat

### zephyr

Do we need proprietary models ? 
Do we really need 13 billion or 70 billion plus models ?

Zephyr , is a model from HuggingFace which is only 7B parameters

In [169]:


chain = ecommerce_prompt | chain_zephyr.bind() 

res =  chain.invoke({"product_title": sample_product_title}) 
rich.print(res)

{
    'question': StringPromptValue(
        text="\nYou are an e-commerce shopping assistant. \nGiven the title of a product, generate two possible 
user search queries. \nIdeally prioritize queries with tokens not in the product title \n\nYour response should be 
a list of comma separated values, eg: `foo, bar, baz`\n\nExamples:   \nInput:   \nTitle: Let's Find Pokémon! 
Special Complete Edition (2nd edition)   \nOutput:    \npokemon book pokedex   \npokemon illustration book   
\nInput:   \nTitle: Keto Comfort Foods: Family Favorite Recipes Made Low-Carb and Healthy   \nOutput:   \nsoul food
cookbook for beginners   \nchicken soup for nutrient \n\nInput:   \nTitle: Biodegradable Bamboo Toothbrush, Natural
Charcoal toothbrushes Soft Bristle Toothbrush Eco-Friendly Sustainable Toothbrush BPA Free Organic Compostable 
Travel Toothbrushes Wooden toothbrushes, 6 Pack\nOutput: \n\n\n"
    ),
    'text': '\n\n1. bamboo toothbrush compostable packaging\n2. eco-friendly toothbrush for 
travel\n\n\nExplanation:\n\n1. The first search query includes "compostable packaging" as a token that is not in 
the product title. This is a common concern for eco-friendly products and can help narrow down search 
results.\n\n2. The second search query includes "travel" as a token that is not in the product title. This can help
users find toothbrushes that are specifically designed for travel, which may be a priority for some users.'
}

In [170]:
print ( res['text'] )



1. bamboo toothbrush compostable packaging
2. eco-friendly toothbrush for travel


Explanation:

1. The first search query includes "compostable packaging" as a token that is not in the product title. This is a common concern for eco-friendly products and can help narrow down search results.

2. The second search query includes "travel" as a token that is not in the product title. This can help users find toothbrushes that are specifically designed for travel, which may be a priority for some users.


## Takeaways

- langchain makes it simple to use many LLMs
- All the LLMs are very good with reasoning
- few shot prompt, can help the response
- Llama chat 13b+ models can perform well
- Even properly trained 7B models, can beat performance 
